In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys
from pathlib import Path

sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../../../electricity_r_data_long.csv")

In [ ]:
df.choice = df.choice.astype(int)

In [ ]:
df[["pf", 'cl', "loc", "wk", "tod", "seas"]] = df[["pf", 'cl', "loc", "wk", "tod", "seas"]].astype(float)

In [ ]:
from choice_learn.data import ChoiceDataset

dataset = ChoiceDataset.from_single_long_df(df=df,
contexts_items_features_columns=["pf", "cl", "loc", "wk", "tod", "seas"],
items_id_column="alt",
contexts_id_column="chid",
choice_mode="one_zero")

In [ ]:
from choice_learn.models.simple_mnl import SimpleMNL

In [ ]:
model = SimpleMNL(optimizer="lbfgs", epochs=100, tolerance=1e-4)
h = model.fit(dataset, sample_weight=www)
h[-1]

In [ ]:
import tensorflow as tf
model.weights = tf.constant([[-0.622498214, -0.108463198, 1.46465945, 1.02686298, -5.48458862, -5.86176395]])

In [ ]:
bigP = model.predict_probas(dataset).numpy()
p = [(bigP[i][dataset.choices[i]]) for i in range(len(dataset))]

In [ ]:
np.min(p)

In [ ]:
from choice_learn.models import ConditionalMNL
from choice_learn.models.base_model import BaseMixtureModel

In [ ]:
mix = BaseMixtureModel(model_class=SimpleMNL,
                          model_parameters={"optimizer": "lbfgs", "epochs": 100, "tolerance": 1e-2},
                          n_latent_classes=3, epochs=200, fit_method="em")

In [ ]:
mix.instantiate()
h1, h2 = mix._em_fit(dataset)

In [ ]:
(mix.models[0].weights,
mix.models[1].weights,
mix.models[2].weights)

In [ ]:
mix.weights

In [ ]:
mix.latent_logits

In [ ]:
h2

In [ ]:
plt.plot(h2)

In [ ]:
len(dataset)

In [ ]:
w1 = [-0.436786, -0.024613, 2.510545, 1.649299, -2.726110, -3.720022]
w2 = [-0.711346, -0.534724, 0.632174, 0.558454, -5.994899, -5.855283]
w3 = [-0.785066, -0.055608, 1.559850, 1.219109, -8.866931, -8.332032]

lp = [np.exp(0), np.exp(-0.515807), np.exp(0.152700)]
lp = np.array(lp) / np.sum(lp)

In [ ]:
mmm = [SimpleMNL(optimizer="lbfgs", epochs=100, tolerance=1e-4) for _ in range(3)]
for _ in mmm:
    _.fit(dataset)

In [ ]:
mmm[0].weights = tf.constant(w1)
mmm[1].weights = tf.constant(w2)
mmm[2].weights = tf.constant(w3)

In [ ]:
probas = [_.predict_probas(dataset).numpy() for _ in mmm]
c_probas = []
for prob in probas:
    prob = [prob[i][dataset.choices[i]] for i in range(len(dataset))]
    c_probas.append(prob)

class_logits = [0, -0.515807, 0.152700]
class_p = [np.exp(l) for l in class_logits]
class_p = np.array(class_p) / np.sum(class_p)
c_probas = np.sum([np.array(c_probas[i]) * class_p[i] for i in range(3)], axis=0)

In [ ]:
np.sum(np.log(c_probas))

In [ ]:
class_p

In [ ]:
np.exp(0), np.exp(-0.515807), np.exp(0.152700)

In [ ]:
probas

In [ ]:
xxx = dataset.contexts_items_features[0]

In [ ]:
p1 = np.exp(np.dot(xxx, w1)) / np.sum(np.exp(np.dot(xxx, w1)), axis=1, keepdims=True)
p2 = np.exp(np.dot(xxx, w2)) / np.sum(np.exp(np.dot(xxx, w2)), axis=1, keepdims=True)
p3 = np.exp(np.dot(xxx, w3)) / np.sum(np.exp(np.dot(xxx, w3)), axis=1, keepdims=True)

In [ ]:
p = np.sum([lp[0] * p1, lp[1] * p2, lp[2] * p3], axis=0)

In [ ]:
nll = [np.log(p[i][z]) for i, z in zip(range(len(p)), dataset.choices)]

In [ ]:
np.sum(nll)

In [ ]:
import tensorflow as tf

tf.keras.metrics.categorical_crossentropy(tf.one_hot(dataset.choices, 4), p)

In [ ]:
tf.reduce_sum(tf.keras.metrics.categorical_crossentropy(tf.one_hot(dataset.choices, 4), p))

In [ ]:
nll[:4]

In [ ]:
np.log(.1), p[0], np.log(p[0][3])

In [ ]:
df.head()